# Premium Allocation Tool

In [ ]:
import numpy as np
import pandas as pd
import json
import uuid  

from pat_back import PatJob, PatFlag, AppSettings, PatHelper

# Get job list 


## Delete Job

In [ ]:
# pd.DataFrame(json.loads(PatJob.get_job_list()))
lst= [a for a in range(100000233,100000300)]
PatHelper.delete(*lst)

## Job parameters

In [ ]:
ret = PatJob.get_job_para(100000197)
if ret:
    print(json.dumps(json.loads(ret),indent=4))


## Result summary 

In [ ]:
df = PatJob.get_summary(100000197)
df

## Results

In [ ]:
df = PatJob.get_results(100000197)
df

## Validation data

In [ ]:
df1,df2,df3 = PatJob.get_validation_data(100000197)
print(len(df1), len(df2), len(df3))

In [ ]:
df2.sort_values(by='LocationIDStack')

# Create New Job 

In [ ]:
job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid']=str(uuid.uuid4())

job = PatJob(job_para)
job.job_id

# Process

In [ ]:
job.perform_analysis()

# Testing

In [ ]:

import json
import numpy as np
import pandas as pd
import pyodbc
from bcpandas import SqlCreds, to_sql

df = pd.read_csv('data/psold_aoi.csv')

creds = SqlCreds(AppSettings.PAT_JOB_SVR, AppSettings.PAT_JOB_DB, AppSettings.PAT_JOB_USR, AppSettings.PAT_JOB_PWD)
to_sql(df,'psold_aoi', creds, index=False, if_exists='append') 

In [ ]:
job_conn = f'''DRIVER={{SQL Server}};Server={AppSettings.PAT_JOB_SVR};Database={AppSettings.PAT_JOB_DB};
            User Id={AppSettings.PAT_JOB_USR};Password={AppSettings.PAT_JOB_PWD};
            MultipleActiveResultSets=true;'''
with pyodbc.connect(job_conn) as conn:
             dfHPR = pd.read_sql_query(f"""select * from psold_hpr order by limit""", conn)
dfHPR

# Start

In [2]:
from datetime import datetime
import numpy as np
import pandas as pd

from pat_back import Psold


In [3]:
aoi_split = np.loadtxt(r"C:\_Working\PAT_20201019\debug\AOI_R.csv")
df_psold = pd.read_csv("C:\_Working\PAT_20201019\debug\ISOt_2020.csv")
df_psold = df_psold[(df_psold.COVG == 4) & (df_psold.SUBGRP == 4)][
    list(('RG','EG','AOCC',*[f'G{i}' for i in range(1, 12)]))]
df_psold.columns=['RG', 'EG', 'OCC', *[f'W{i}' for i in range(1, 12)]]

psold  = Psold(df_psold, aoi_split, 1.05, datetime(2020,12,31))


df_wts = pd.read_csv(r"C:\_Working\PAT_20201019\debug\psold_weight.csv").set_index('RG')
df_wts.loc[1:39,['PremiumWeight']] = 0
df_wts.loc[39,['PremiumWeight']] = 1

DT = pd.read_csv("C:\_Working\PAT_20201019\debug\dt_2020.csv").iloc[:,:10].rename(columns={
            'polID': 'PolicyID',
            'rawTIV' : 'TIV',
            'limit' : 'Limit',
            'prem' : 'PolPrem', 
            'SIRD':'Retention',
            'LR': 'LossRatio',
            'stack': 'LocStack',
            'rtG': 'RatingGroup'
        })
DT.reset_index(inplace= True, drop=True)
DT.reset_index(inplace=True)

# # df_hpr = pd.read_csv("C:\_Working\PAT_20201019\debug\HPRBlend.csv") if hpr else None

# # DT = DT.sort_values('polDed')#.reset_index(drop=True)
df = psold.pat(DT, df_wts, avg_acc_date=datetime(2023,1,1), addt_cvg=0.5)
df.set_index('index').sort_index()[['locID','AllocPrem']]


,locID,AllocPrem
index,,
0,1,2.301891e+05
1,2,2.527544e+05
2,3,7.781362e+05
3,4,0.000000e+00
4,5,8.269253e+04
5,6,2.133151e+06
6,7,4.592198e+06
7,8,3.374567e+05
8,9,NaN


In [3]:
df_hpr = pd.read_csv("C:\_Working\PAT_20201019\debug\HPRBlend.csv")
df_hpr

,Limit,Weight
0,0.000000e+00,0.00
1,2.000000e+07,0.00
2,3.000000e+07,0.25
3,5.000000e+07,0.50
4,1.000000e+08,1.00
5,1.000000e+33,1.00
